In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import tensorflow as tf
# Load the CSV file and specify column names
df = pd.read_csv('/content/drive/My Drive/final_preprocessed_data_yidong_devansh.csv',
                 header=None, names=['statement', 'analysis'])

df['analysis'] = float(df['analysis'].iloc[0])
print(df)
# Print the first few rows to check
# print(df)
# !cat /content/drive/My\ Drive/foo.txt

                                                statement  analysis
0       denial of normal the con be asked to comment o...       1.0
1       just by being able to tweet this insufferable ...       1.0
2       that is retarded you too cute to be single tha...       1.0
3       thought of a real badass mongol style declarat...       1.0
4                                     afro american basho       1.0
...                                                   ...       ...
451704  crash another movie from left field i have to ...       1.0
451705  i why do not you debate first before starting ...       1.0
451706  removal of i reverted the removal of the above...       1.0
451707  i have unblocked you eddie as i discussed on u...       1.0
451708  you have the ability to delete that revision a...       1.0

[451709 rows x 2 columns]


In [4]:
import torch
torch.cuda.is_available()

!pip install -U datasets
from datasets import load_dataset, Dataset
!pip install -U evaluate
!pip install -U accelerate
!pip install -U transformers

import numpy as np
import pandas as pd
import evaluate
import accelerate
from transformers import AutoTokenizer, pipeline
from transformers import (BertTokenizerFast,TFBertTokenizer,BertTokenizer,RobertaTokenizerFast,
                          DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,
                          TFBertModel,create_optimizer)


  Using cached transformers-4.51.2-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.51.2-py3-none-any.whl (10.4 MB)


In [5]:
dataset = Dataset.from_pandas(df)
# print(new_dataset)
# small_dataset = new_dataset.select([0, 10, 20, 30, 40, 45000])
# metrics_list = evaluate.list_evaluation_modules()
# print(small_dataset)
# print(small_dataset['statement'])
# print(small_dataset['analysis'])

In [6]:
#load model
model_id="bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
#test
test_input_1='The Weather of Today is Gréat! zwp'
test_input_2='How are you doing?'
inputs=[test_input_1,test_input_2]

tokenizer.tokenize(inputs)
output=tokenizer(inputs,padding=True,truncation=True,max_length=128)
print(output)
tokenizer.decode(output['input_ids'][0])

{'input_ids': [[101, 1996, 4633, 1997, 2651, 2003, 2307, 999, 1062, 2860, 2361, 102], [101, 2129, 2024, 2017, 2725, 1029, 102, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]}


'[CLS] the weather of today is great! zwp [SEP]'

In [8]:
def preprocess_function(examples):
  return tokenizer(examples["statement"],padding=True,truncation=True,)


In [8]:
pd.DataFrame({
    'statement':tokenized_dataset[:5]['statement'],
    'analysis':tokenized_dataset[:5]['analysis'],
    'input_ids':tokenized_dataset[:5]['input_ids'],
    'token_type_ids':tokenized_dataset[:5]['token_type_ids'],
    'attention_mask':tokenized_dataset[:5]['attention_mask']
    })
!pip show tensorflow

NameError: name 'tokenized_dataset' is not defined

In [9]:
split_dataset = dataset.train_test_split(test_size=0.1)
tokenized_dataset = split_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/406538 [00:00<?, ? examples/s]

Map:   0%|          | 0/45171 [00:00<?, ? examples/s]

In [10]:
pd.DataFrame({
    'statement':tokenized_dataset['train'][:5]['statement'],
    'analysis':tokenized_dataset['train'][:5]['analysis'],
    'input_ids':tokenized_dataset['train'][:5]['input_ids'],
    'token_type_ids':tokenized_dataset['train'][:5]['token_type_ids'],
    'attention_mask':tokenized_dataset['train'][:5]['attention_mask']
    })

print(tokenized_dataset['train'])

Dataset({
    features: ['statement', 'analysis', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 406538
})


In [11]:
BATCH_SIZE = 8
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask', 'analysis'],
    shuffle=True,
    batch_size=BATCH_SIZE,
)

# Test Dataset

tf_val_dataset = tokenized_dataset["test"].to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask', 'analysis'],
    shuffle=True,
    batch_size=BATCH_SIZE,
    #collate_fn=data_collator
)

In [12]:
def swap_positions(dataset):
  return {'input_ids':dataset['input_ids'],
          'token_type_ids':dataset['token_type_ids'],
          'attention_mask':dataset['attention_mask'],},dataset['analysis']

tf_train_dataset=tf_train_dataset.map(swap_positions).prefetch(tf.data.AUTOTUNE)
tf_val_dataset=tf_val_dataset.map(swap_positions).prefetch(tf.data.AUTOTUNE)

In [13]:
for i in tf_train_dataset.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[  101,  2632,  1053, ...,     0,     0,     0],
       [  101,  1045,  2064, ...,     0,     0,     0],
       [  101, 10443,  2026, ...,     0,     0,     0],
       ...,
       [  101,  2144,  2043, ...,     0,     0,     0],
       [  101,  2025,  8010, ...,     0,     0,     0],
       [  101,  2066,  2081, ...,     0,     0,     0]])>, 'token_type_ids': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>}, <tf.Tensor: shape=(8,), dtype=float32, 

In [14]:
model=TFBertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=1)
model.summary()

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  769       
                                                                 
Total params: 109483009 (417.64 MB)
Trainable params: 109483009 (417.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
#training
num_epochs = 3
batches_per_epoch = len(tokenized_dataset["train"]) // BATCH_SIZE
total_train_steps = int(batches_per_epoch * num_epochs)

In [16]:
# Optimizer
optimizer, schedule = create_optimizer(init_lr=2e-5,num_warmup_steps=0, num_train_steps=total_train_steps)

In [17]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=optimizer,
    metrics=['accuracy'],)
    #run_eagerly=True)

In [18]:
#fitting
history=model.fit(
    tf_train_dataset.take(1000),
    validation_data=tf_val_dataset,
    epochs=3)

Epoch 1/3


FailedPreconditionError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_199 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.1

In [20]:
!echo  $LD_LIBRARY_PATH
!which colabfold

/usr/local/lib/python3.11/dist-packages/cv2/../../lib64:/usr/lib64-nvidia


In [ ]:
df['analysis'] = float(df['analysis'].iloc[0])
print(df)

                                                statement  analysis
0       denial of normal the con be asked to comment o...       1.0
1       just by being able to tweet this insufferable ...       1.0
2       that is retarded you too cute to be single tha...       1.0
3       thought of a real badass mongol style declarat...       1.0
4                                     afro american basho       1.0
...                                                   ...       ...
451704  crash another movie from left field i have to ...       1.0
451705  i why do not you debate first before starting ...       1.0
451706  removal of i reverted the removal of the above...       1.0
451707  i have unblocked you eddie as i discussed on u...       1.0
451708  you have the ability to delete that revision a...       1.0

[451709 rows x 2 columns]


In [20]:
!pip install --upgrade "jax[cuda12]" -f https://storage.googleapis.com/jax-releases/jax_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/jax_releases.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 24.6 MB/s eta 0:00:00
  Attempting uninstall: jax-cuda12-pjrt
    Found existing installation: jax-cuda12-pjrt 0.5.1
    Uninstalling jax-cuda12-pjrt-0.5.1:
      Successfully uninstalled jax-cuda12-pjrt-0.5.1
  Attempting uninstall: nvidia-cuda-nvcc-cu12
    Found existing installation: nvidia-cuda-nvcc-cu12 12.5.82
    Uninstalling nvidia-cuda-nvcc-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-nvcc-cu12-12.5.82
  Attempting uninstall: jax-cuda12-plugin
    Found existing installation: jax-cuda12-plugin 0.5.1
    Uninstalling j

In [ ]:
import tensorflow_datasets as tfds # Tf Datasets
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (Dense,Flatten,InputLayer,BatchNormalization,
                                     Dropout,Input,LayerNormalization)
from tensorflow.keras.losses import (BinaryCrossentropy,CategoricalCrossentropy,
                                    SparseCategoricalCrossentropy)
from tensorflow.keras.metrics import (Accuracy,TopKCategoricalAccuracy,
                                 CategoricalAccuracy, SparseCategoricalAccuracy)
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import (BertTokenizerFast,TFBertTokenizer,BertTokenizer,RobertaTokenizerFast,
                          DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,
                          TFBertModel,create_optimizer)

ModuleNotFoundError: No module named 'tensorflow.compat.v2.keras'

In [ ]:
#loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
#accuracy plot
!pip list | grep tensorflow
print(tf.config.list_physical_devices('GPU'))
!nvcc --version
!pip3 install --user tensorflow==2.0


tensorflow                         2.18.0
tensorflow-datasets                4.9.8
tensorflow-hub                     0.16.1
tensorflow-io-gcs-filesystem       0.37.1
tensorflow-metadata                1.16.1
tensorflow-probability             0.25.0
tensorflow-text                    2.18.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
ERROR: Could not find a version that satisfies the requirement tensorflow==2.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribu

In [ ]:
!nvidia-smi
!grep CUDNN_MAJOR -A 2 /usr/local/cuda/include/cudnn.h
!nvcc  --version
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2
print('CUDA:',torch.version.cuda)

cudnn = torch.backends.cudnn.version()
print(cudnn)
cudnn_major = cudnn // 1000
cudnn = cudnn % 1000
cudnn_minor = cudnn // 100
cudnn_patch = cudnn % 100
print( 'cuDNN:', '.'.join([str(cudnn_major),str(cudnn_minor),str(cudnn_patch)]) )

Wed Apr  2 07:51:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P0             32W /   70W |    8310MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!sudo apt remove --purge '^cuda.*' '^nvidia.*' '^cudnn.*'
!sudo apt autoremove
!sudo apt clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-11-0' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-11-1' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-11-7' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-11-8' for regex '^cuda.*'
Note, selecting 'cuda-toolkit-12-5-config-common' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-12-0' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-12-1' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-12-2' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-12-3' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-12-4' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-12-5' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-12-6' for regex '^cuda.*'
Note, selecting 'cuda-cudart-dev-12-8' for regex '^cuda.*'
Note, selecting 'cuda-toolkit-12-6-config-common' for r

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/11.2.2/local_installers/cuda_11.2.2_460.32.03_linux.run
!sudo sh cuda_11.2.2_460.32.03_linux.run --silent --toolkit

--2025-04-02 07:56:25--  https://developer.download.nvidia.com/compute/cuda/11.2.2/local_installers/cuda_11.2.2_460.32.03_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.52.40.64, 23.52.40.50
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.52.40.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3057439068 (2.8G) [application/octet-stream]
Saving to: ‘cuda_11.2.2_460.32.03_linux.run’

cuda_11.2.2_460.32. 100%[===================>]   2.85G  27.6MB/s    in 44s     

2025-04-02 07:57:09 (66.1 MB/s) - ‘cuda_11.2.2_460.32.03_linux.run’ saved [3057439068/3057439068]

 Failed to verify gcc version. See log at /var/log/cuda-installer.log for details.


In [ ]:
!wget https://developer.nvidia.com/compute/machine-learning/cudnn/secure/8.1.0/local_installers/cudnn-8.1-linux-x64-v8.1.0.77.tgz
!tar -xzvf cudnn-8.1-linux-x64-v8.1.0.77.tgz
!sudo cp cuda/include/cudnn*.h /usr/local/cuda/include
!sudo cp cuda/lib64/libcudnn* /usr/local/cuda/lib64
!sudo chmod a+r /usr/local/cuda/include/cudnn*.h /usr/local/cuda/lib64/libcudnn*


--2025-04-02 07:59:32--  https://developer.nvidia.com/compute/machine-learning/cudnn/secure/8.1.0/local_installers/cudnn-8.1-linux-x64-v8.1.0.77.tgz
Resolving developer.nvidia.com (developer.nvidia.com)... 23.52.40.50, 23.52.40.64
Connecting to developer.nvidia.com (developer.nvidia.com)|23.52.40.50|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-04-02 07:59:33 ERROR 404: Not Found.

tar (child): cudnn-8.1-linux-x64-v8.1.0.77.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
cp: cannot stat 'cuda/include/cudnn*.h': No such file or directory
cp: cannot stat 'cuda/lib64/libcudnn*': No such file or directory
chmod: cannot access '/usr/local/cuda/include/cudnn*.h': No such file or directory
chmod: cannot access '/usr/local/cuda/lib64/libcudnn*': No such file or directory


In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Wed Apr  2 07:59:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P0       